Lesson 3 Graph-theoretic models

Applications for building networks:
* Roads
* computer networks
* Financial networks
* Political networks
* Criminal networks (buelltin board, red rope)
* Social networks

We can look at paths, and connectivity of everything

Types of problems:
* Finding links
* Finding shortest path
* Partition a grpah into sets of cennected elements
* Finding the most efficient way to separate sets fo connected elements

Types of optimization
* Shortest path
* Fastest avg speed
* Expected time to get to and from

Building a graphs nodes and objects in a class

In [3]:
class Node(object):
    def __init__(self, name): #need to create instances of nodes, which have names
        """Assumes name is a string"""
        self.name = name
    def getName(self):
        return self.name
    def __str__(self):
        return self.name

In [6]:
class Edge(object):
    def __init__(self, src, dest): #need to create instances of edges, which are nodes 
        """Assumes src and dest are nodes"""
        self.src = src #this will be a source node
        self.dest = dest #this will be a source destination
    def getSource(self):
        return self.src #retuning the source node
    def getDestination(self):
        return self.dest #returning the source destination
    def __str__(self): 
        return self.src.getName() + '->' + self.dest.getName() #


In [12]:
# Edge object can still use the getName method of a Node object 
# because the Edge class takes src and dest parameters, which are instances of the Node class.
# quick look at non explicit class inheritance, and how Edges can call getName

# Create nodes
node1 = Node("A")
node2 = Node("B")
print(node1)
print(node1.getName())

# Create an edge using the nodes
edge = Edge(node1, node2)

# Use the getName method of the Node class through the Edge class
print(edge.getSource().getName())  # Output: A
print(edge.getDestination().getName())  # Output: B

# You can also use the __str__ method of the Edge class, which calls getName internally
print(edge)

A
A
A
B
A->B


In [14]:
# Creating an agacency list, for every node, a list of destinations with a single edge
class Digraph(object):
    """when you put a node in graph, it will be the key to a dict, 
    edges is a dict mapping each node to a lsit of its children, aka where they are linked
    Nodes are the keys,and edges are the values"""

    def __init__(self): #initialize a graph and goal is to have a list of nodes that an edge can reach
        self.edges = {}
    
    def addNode(self, node): #add nodes to dictionary to the dictionary, with values as empty lists
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            ''' it adds the node to the edges dictionary with an empty list as its value. 
            This list will eventually store the children (neighbors) of the node.'''
            self.edges[node] = [] 
    
    def addEdge(self, edge): # 
        src = edge.getSource() 
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges): #checking edges are in the graph / dictionary
            raise ValueError('Node not in graph')
        self.edges[src].append(dest) #indexing into the dictionary, and appending to the list
    
    def childrenOf(self, node): #returns all children dest of a node
            return self.edges[node] #indexing on the node
    
    def hasNode(self, node): # check if a node is in a graph
        return node in self.edges
    
    def getNode(self, name): #return node if in dict
        for n in self.edges: # looks at nodes in graph dict
            if n.getName() == name: #call getName() on the nodes, which returns name as a string, and compare to parameter
                return n 
        raise NameError(name) 

    '''calling the print statement will loop over the nodes in the graph / key in dictionary, and return add the source and destination of each   '''
    def __str__(self): 
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1] #omit final newline

In [15]:
class Graph(Digraph): #add the edge reverse as well
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

In [16]:
def buildCityGraph(graphType):
    g = graphType()
    for name in ('Boston', 'Providence', 'New York', 'Chicago',
                 'Denver', 'Phoenix', 'Los Angeles'): #Create 7 nodes
        g.addNode(Node(name))
    '''
    First run g.getNode(), which returns all values in the dict assigned to the node boston
    on the first getNode it will return a empty list, same with Providence.
    Then it runs edge() with node parameters Boston and Providence, initializing boston as source and Prov as destination
    Then runs addEdge() with the new edge of Bos and Prov, returning nodes for each as sources and destinations
    addEdge then adds Providence to the dictionary of edges, where Boston is a key and Providence is a value, and calling
    Boston will return all the destinations (values) they are connected to. 
    '''    
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('Providence'))) 
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('Boston')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('New York'), g.getNode('Chicago')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Denver')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Los Angeles'), g.getNode('Boston')))
    return g

In [17]:
g = buildCityGraph(Digraph)
print(g)

Boston->Providence
Boston->New York
Providence->Boston
Providence->New York
New York->Chicago
Chicago->Denver
Chicago->Phoenix
Denver->Phoenix
Denver->New York
Los Angeles->Boston


In [21]:
def printPath(path):
    """Assumes path is a list of nodes"""
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) - 1:
            result = result + '->'
    return result 

def DFS(graph, start, end, path, shortest, toPrint = False): #depth-first search recursive
    """Assumes graph is a Digraph; start and end are nodes;
          path and shortest are lists of nodes
       Returns a shortest path from start to end in graph
        '''If a new path (newPath) is found from the child to the end node, 
            and it is shorter than the current shortest path, it updates the shortest path.'''
       """
    path = path + [start] # path is an empty list, first time around gives start node
    if toPrint:
        print('Current DFS path:', printPath(path))
    if start == end: #base case, see if trip is done
        return path
    for node in graph.childrenOf(start): #loop over the list of children in the start node. The edges I can reach
        if node not in path: #avoid looping over itself
            if shortest == None or len(path) < len(shortest): 
                newPath = DFS(graph, node, end, path, shortest, toPrint) #recursivly start looking forpath at next node
                if newPath != None: #if there is a new path, then update shortest path with this newPath
                    shortest = newPath
        elif toPrint: # if node already in path
            print('Already visited', node)
    return shortest
    
def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return DFS(graph, start, end, [], None, toPrint)

def testSP(source, destination):
    g = buildCityGraph(Digraph)
    sp = shortestPath(g, g.getNode(source), g.getNode(destination),
                      toPrint = True)
    if sp != None: 
        print('Shortest path from', source, 'to',
              destination, 'is', printPath(sp))
    else: #if there was no path returned
        print('There is no path from', source, 'to', destination)

# testSP('Chicago', 'Boston')
# print()
testSP('Boston', 'Phoenix')
print()

Current DFS path: Boston
Current DFS path: Boston->Providence
Already visited Boston
Current DFS path: Boston->Providence->New York
Current DFS path: Boston->Providence->New York->Chicago
Current DFS path: Boston->Providence->New York->Chicago->Denver
Current DFS path: Boston->Providence->New York->Chicago->Denver->Phoenix
Already visited New York
Current DFS path: Boston->Providence->New York->Chicago->Phoenix
Current DFS path: Boston->New York
Current DFS path: Boston->New York->Chicago
Current DFS path: Boston->New York->Chicago->Denver
Current DFS path: Boston->New York->Chicago->Denver->Phoenix
Already visited New York
Current DFS path: Boston->New York->Chicago->Phoenix
Shortest path from Boston to Phoenix is Boston->New York->Chicago->Phoenix



In [ ]:
printQueue = True 

def BFS(graph, start, end, toPrint = False): # Breath first search
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    initPath = [start]
    pathQueue = [initPath]
    while len(pathQueue) != 0:
        #Get and remove oldest element in pathQueue
        if printQueue:
            print('Queue:', len(pathQueue))
            for p in pathQueue:
                print(printPath(p))
        tmpPath = pathQueue.pop(0)
        if toPrint:
            print('Current BFS path:', printPath(tmpPath))
            print()
        lastNode = tmpPath[-1]
        if lastNode == end:
            return tmpPath
        for nextNode in graph.childrenOf(lastNode):
            if nextNode not in tmpPath:
                newPath = tmpPath + [nextNode]
                pathQueue.append(newPath)
    return None

def shortestPath(graph, start, end, toPrint = False):
    """Assumes graph is a Digraph; start and end are nodes
       Returns a shortest path from start to end in graph"""
    return BFS(graph, start, end, toPrint)
    
#testSP('Boston', 'Phoenix')
    